In [78]:
import numpy as np
import pandas as pd


In [79]:
df=pd.read_csv("/kaggle/input/uci-air-quality-dataset/AirQualityUCI.csv")

In [80]:
df=df.drop(columns=["Unnamed: 15","Unnamed: 16","Time"])
df=df.dropna(how='any',axis=0)
df

,Date,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,3/10/2004,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,3/10/2004,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,3/10/2004,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,3/10/2004,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,3.1,1314.0,-200.0,13.5,1101.0,472.0,539.0,190.0,1374.0,1729.0,21.9,29.3,0.7568
9353,4/4/2005,2.4,1163.0,-200.0,11.4,1027.0,353.0,604.0,179.0,1264.0,1269.0,24.3,23.7,0.7119
9354,4/4/2005,2.4,1142.0,-200.0,12.4,1063.0,293.0,603.0,175.0,1241.0,1092.0,26.9,18.3,0.6406
9355,4/4/2005,2.1,1003.0,-200.0,9.5,961.0,235.0,702.0,156.0,1041.0,770.0,28.3,13.5,0.5139


In [81]:
df.isnull().sum()

Date             0
CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
dtype: int64

In [82]:
df["Date"]=pd.to_datetime(df["Date"])
df["Year"]=df["Date"].dt.year
df["Month"]=df["Date"].dt.month
df["Day"]=df["Date"].dt.day
df=df.drop(columns=["Date"])
df.isnull().sum()

CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
Year             0
Month            0
Day              0
dtype: int64

In [83]:
df.dtypes

CO(GT)           float64
PT08.S1(CO)      float64
NMHC(GT)         float64
C6H6(GT)         float64
PT08.S2(NMHC)    float64
NOx(GT)          float64
PT08.S3(NOx)     float64
NO2(GT)          float64
PT08.S4(NO2)     float64
PT08.S5(O3)      float64
T                float64
RH               float64
AH               float64
Year               int32
Month              int32
Day                int32
dtype: object

In [95]:
df_reshaped=df.values
df_reshaped=np.repeat(df_reshaped[:,np.newaxis,:],3,axis=1).astype(np.float32)
df_reshaped.shape

(9357, 3, 16)

# # AutoEncoder

In [85]:
from keras.layers import Layer,Dense,LSTM,GRU,SimpleRNN,BatchNormalization,Dropout,Input,ReLU,LeakyReLU
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [86]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [96]:
inputs=Input(shape=(3,16))
E=SimpleRNN(32,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=SimpleRNN(64,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=LSTM(128,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=LSTM(256,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=GRU(556,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=GRU(1056,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=GRU(2056,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=GRU(1056,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=GRU(556,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=LSTM(256,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=LSTM(128,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=SimpleRNN(64,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=LeakyReLU(0.08)(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=SimpleRNN(32,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)
outputs=SimpleRNN(1,return_sequences=True,activation="sigmoid")(E)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 3, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_48 (SimpleRNN)       │ (None, 3, 32)          │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_112 (Dropout)           │ (None, 3, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_112         │ (None, 3, 32)          │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_49 (SimpleRNN)       │ (None, 3, 64)          │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_94 (LeakyReLU)      │ (None, 3, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_113 (Dropout)           │ (None, 3, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_113         │ (None, 3, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_32 (LSTM)                  │ (None, 3, 128)         │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_95 (LeakyReLU)      │ (None, 3, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_114 (Dropout)           │ (None, 3, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_114         │ (None, 3, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_33 (LSTM)                  │ (None, 3, 256)         │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_96 (LeakyReLU)      │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_115 (Dropout)           │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_115         │ (None, 3, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_40 (GRU)                    │ (None, 3, 556)         │     1,357,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_97 (LeakyReLU)      │ (None, 3, 556)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_116 (Dropout)           │ (None, 3, 556)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_116         │ (None, 3, 556)         │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_41 (GRU)                    │ (None, 3, 1056)        │     5,113,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_98 (LeakyReLU)      │ (None, 3, 1056)        │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 39,806,274 (151.85 MB)

 Trainable params: 39,793,794 (151.80 MB)

 Non-trainable params: 12,480 (48.75 KB)

In [97]:
model.fit(df_reshaped,df_reshaped,epochs=50,batch_size=32,validation_split=(0.2),callbacks=[early_stopping])

Epoch 1/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 184s 669ms/step - accuracy: 0.0041 - loss: 665210.3750 - val_accuracy: 0.0186 - val_loss: 578406.5625
Epoch 2/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 150s 642ms/step - accuracy: 0.0066 - loss: 666988.0625 - val_accuracy: 0.0186 - val_loss: 578395.6250
Epoch 3/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 151s 644ms/step - accuracy: 0.0072 - loss: 662366.5625 - val_accuracy: 0.0186 - val_loss: 578380.6875
Epoch 4/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 153s 652ms/step - accuracy: 0.0071 - loss: 667976.3750 - val_accuracy: 0.0186 - val_loss: 578362.7500
Epoch 5/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 203s 655ms/step - accuracy: 0.0071 - loss: 664700.5625 - val_accuracy: 0.0186 - val_loss: 578343.2500
Epoch 6/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 201s 653ms/step - accuracy: 0.0067 - loss: 664602.2500 - val_accuracy: 0.0186 - val_loss: 578323.4375
Epoch 7/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 202s 655ms/step - accuracy: 0.0067 - loss: 662954.8125 - val_accuracy: 0.0186 - val_loss: 578304.8125
Epoch 

# # QLSTM

In [ ]:
def build_quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(len(data))]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        circuit.append(cirq.rx(value*np.pi).on(qubit[i]))
        circuit.append(cirq.ry(value*np.pi).on(qubit[i]))
        circuit.append(cirq.rz(value*np.pi).on(qubit[i]))
    circuit.append([cirq.CNOT(qubit[i],qubit[i+1]) for i in range(len(data)-1)])
    circuit.append(cirq.measure(qubit[i],key=f"qubit_{i}" for i in range(len(data)))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    for input_data in data:
        circuit,qubit=build_quantum_circuit()
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        for i in range(len(input_data)):
            measure0=result.measurements(f'qubit_{i}').flatten()[0]
            results.append(measure0)
    return np.array(results,dtype=np.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(Quantumlayer,self).__init__()
    def call(self,inputs):
        inputs_flat=tf.repeat(inputs,[-1,inputs.shape[-1]])
        output=tf.numpy_function(run_quantum_circuit,[inputs_flat],tf.float32)
        output.shape(inputs.shape)
        return output
model2=Sequential([
    Quantumlayer(),
    Flatten(),
    Dense(32,activation="relu"),
    Dense(1,activation="sigmoid")
])
model2.compile(optimizer='adam',loss='mse',metics=['mae'])
model2.summary()

In [ ]:
model2.fit(df_reshaped,df_reshaped,epochs=50,batch_size=32,validation_split=(0.2),callbacks=[early_stopping])